# Comparação de gêneros no OBRAS entre PALAVRAS e UDPIPE

## Abrindo corpoPALAVRAS e corpoOBRAS

In [109]:
import re
with open("corpoPALAVRAS", encoding="latin-1") as f:
    PALAVRAS = f.read()
with open("corpoOBRAS", encoding="latin-1") as f:
    OBRAS = f.read()

## Criando estrutura de dados

In [110]:
PALAVRAS_obras = [{
        'obra': re.findall('<obra id="([^"]*)"', x),
        'tituloobra': re.findall('<tituloobra id="([^"]*)"', x),
        'autor': re.findall('<autor id="([^"]*)"', x),
        'sentences': [[{
                    'forma': z.split("\t")[0],
                    'lema': z.split("\t")[1],
                    'pos': z.split("\t")[2],
                    'genero': z.split("\t")[5]
                } for z in y.split("\n") if len(z.split("\t")) > 5
                      ] for y in re.findall('<u>(.*?)</u>', x, flags=re.DOTALL) if '\t' in y]
        } for x in re.findall('<obra.*?</obra>', PALAVRAS, flags=re.DOTALL)]

print(PALAVRAS_obras[0]['autor'])
print(PALAVRAS_obras[0]['obra'])
print(PALAVRAS_obras[0]['tituloobra'])
PALAVRAS_obras[0]['sentences'][0]

['Aluísio Azevedo']
['A_Mortalha_de_Alzira Prosa:romance AA 1894 naturalismo_realismo_romantismo masc ']
['A Mortalha de Alzira']


[{'forma': 'A', 'lema': 'o', 'pos': 'DET_artd', 'genero': 'F'},
 {'forma': 'MORTALHA', 'lema': 'mORTALHA', 'pos': 'N', 'genero': 'F'},
 {'forma': 'DE', 'lema': 'de', 'pos': 'PRP', 'genero': '0'},
 {'forma': 'ALZIRA', 'lema': 'Alzira', 'pos': 'PROP', 'genero': 'F'}]

In [111]:
OBRAS_obras = [{
        'obra': re.findall('<obra id="([^"]*)"', x),
        'tituloobra': re.findall('<tituloobra id="([^"]*)"', x),
        'autor': re.findall('<autor id="([^"]*)"', x),
        'sentences': [[{
                    'forma': z.split("\t")[0],
                    'lema': z.split("\t")[8],
                    'pos': z.split("\t")[9],
                    'genero': z.split("\t")[12]
                } for z in y.split("\n") if len(z.split("\t")) > 5
                      ] for y in re.findall('<u>(.*?)</u>', x, flags=re.DOTALL) if '\t' in y]
        } for x in re.findall('<obra.*?</obra>', OBRAS, flags=re.DOTALL)]

print(OBRAS_obras[0]['autor'])
print(OBRAS_obras[0]['obra'])
print(OBRAS_obras[0]['tituloobra'])
OBRAS_obras[0]['sentences'][0]

['Aluísio Azevedo']
['A_Mortalha_de_Alzira Prosa:romance AA 1894 naturalismo_realismo_romantismo masc ']
['A Mortalha de Alzira']


[{'forma': 'A', 'lema': 'o', 'pos': 'DET_artd', 'genero': 'F'},
 {'forma': 'MORTALHA', 'lema': 'mORTALHA', 'pos': 'N', 'genero': 'F'},
 {'forma': 'DE', 'lema': 'de', 'pos': 'PRP', 'genero': '0'},
 {'forma': 'ALZIRA', 'lema': 'Alzira', 'pos': 'PROP', 'genero': 'F'}]

## Alinhando PALAVRAS e OBRAS

### Número de obras no PALAVRAS e no OBRAS

In [112]:
print(len(PALAVRAS_obras))
print(len(OBRAS_obras))

311
311


### Buscar obras que existem em um e não no outro

In [113]:
obras_proibidas = [x for x in [y['tituloobra'] for y in PALAVRAS_obras] if x not in [z['tituloobra'] for z in OBRAS_obras]]
obras_proibidas += [x for x in [y['tituloobra'] for y in OBRAS_obras] if x not in [z['tituloobra'] for z in PALAVRAS_obras]]
print([x for x in [y['tituloobra'] for y in PALAVRAS_obras] if x not in [z['tituloobra'] for z in OBRAS_obras]])
print([x for x in [y['tituloobra'] for y in OBRAS_obras] if x not in [z['tituloobra'] for z in PALAVRAS_obras]])
PALAVRAS_obras = [x for x in PALAVRAS_obras if x['tituloobra'] not in obras_proibidas]
OBRAS_obras = [x for x in OBRAS_obras if x['tituloobra'] not in obras_proibidas]

print(len(PALAVRAS_obras))
print(len(OBRAS_obras))

[]
[]
311
311


### Buscar obras fora de ordem entre PALAVRAS e OBRAS

In [114]:
fora_de_ordem = False
for i in range(len(PALAVRAS_obras)):
    if PALAVRAS_obras[i]['tituloobra'] != OBRAS_obras[i]['tituloobra']:
        fora_de_ordem = True
        print(f"{PALAVRAS_obras[i]['tituloobra']} / {OBRAS_obras[i]['tituloobra']}")
        
if not fora_de_ordem:
    print("Não há obras fora de ordem")

Não há obras fora de ordem


### Buscar obras sem id

In [115]:
obras_proibidas = []
for obra in PALAVRAS_obras:
    if not obra['obra']:
        print(obra['tituloobra'])
        obras_proibidas.append(obra['tituloobra'])        

PALAVRAS_obras = [x for x in PALAVRAS_obras if x['tituloobra'] not in obras_proibidas]
OBRAS_obras = [x for x in OBRAS_obras if x['tituloobra'] not in obras_proibidas]

print(len(PALAVRAS_obras))
print(len(OBRAS_obras))

['Eterno!']
['Orai por ele!']
['Pobre Finoca!']
['Suje-se gordo!']
['Vênus! Divina Vênus!']
['Vinte Anos! Vinte Anos!']
['Viver!']
304
304


### Buscar obras com número de sentenças diferente

In [116]:
sentencas_diferentes = False
for i in range(len(PALAVRAS_obras)):
    if len(PALAVRAS_obras[i]['sentences']) != len(OBRAS_obras[i]['sentences']):
        sentencas_diferentes = True
        print(f"{i} - {PALAVRAS_obras[i]['tituloobra']}")
        print(f"PALAVRAS ({len(PALAVRAS_obras[i]['sentences'])}) / ACDC ({len(OBRAS_obras[i]['sentences'])})")
        
if not sentencas_diferentes:
    print('Não há obras com número de sentenças diferente')

Não há obras com número de sentenças diferente


### Buscar sentenças com números de tokens diferentes e eliminar

In [118]:
output = ""
sentences_inequal = {}
diff_list = {'OBRAS': {}, 'PALAVRAS': {}}
n_sentences = 0
for i in range(len(PALAVRAS_obras)):
    sentences_inequal[i] = []
    for k in range(len(OBRAS_obras[i]['sentences'])):
        text_PALAVRAS = " ".join([x['forma'] for x in PALAVRAS_obras[i]['sentences'][k]])
        text_OBRAS = " ".join([x['forma'] for x in OBRAS_obras[i]['sentences'][k]])
        if len(text_PALAVRAS.split()) != len(text_OBRAS.split()):
            output += f"{i}-{k}:"
            output += "\n" + text_PALAVRAS
            output += "\n" + text_OBRAS
            output += "\n\n"
            lista_PALAVRAS = [x['forma'] for x in PALAVRAS_obras[i]['sentences'][k]]
            lista_OBRAS = [x['forma'] for x in OBRAS_obras[i]['sentences'][k]]
            for diff in lista_PALAVRAS:
                if diff not in lista_OBRAS:
                    diff_list['OBRAS'][diff] = diff_list['OBRAS'][diff] + 1 if diff in diff_list['OBRAS'] else 1
            for diff in lista_OBRAS:
                if diff not in lista_PALAVRAS:
                    diff_list['PALAVRAS'][diff] = diff_list['PALAVRAS'][diff] + 1 if diff in diff_list['PALAVRAS'] else 1
            sentences_inequal[i].append(k)

for i in range(len(PALAVRAS_obras)):
    PALAVRAS_obras[i]['sentences'] = [x for k, x in enumerate(PALAVRAS_obras[i]['sentences']) if k not in sentences_inequal[i]]
    OBRAS_obras[i]['sentences'] = [x for k, x in enumerate(OBRAS_obras[i]['sentences']) if k not in sentences_inequal[i]]

import pprint
print(f"Total de sentenças no corpus: {sum([len(x) for x in OBRAS_obras[i]['sentences'] for i in range(len(OBRAS_obras))])}")
print(f"{len([x for x in output.splitlines() if x])/2} sentenças com tamanho diferente, que foram suprimidas do experimento (salvas em lista_suprimidas.txt)")
print(f"Total de sentenças no corpus sem as suprimidas: {sum([len(x) for x in OBRAS_obras[i]['sentences'] for i in range(len(OBRAS_obras))])}")
for corpus in diff_list:
    print(f"\n{len(diff_list[corpus])} palavras que foram suprimidas de {corpus}\n")
    pprint.pprint(sorted(diff_list[corpus].items(), key=lambda x: x[1], reverse=True))
print(f"\n{output}")
with open("lista_suprimidas.txt", "w") as f:
    f.write(output)

Total de sentenças no corpus: 12997520
366.0 sentenças com tamanho diferente, que foram suprimidas do experimento (salvas em lista_suprimidas.txt)
Total de sentenças no corpus sem as suprimidas: 12997520

35 palavras que foram suprimidas de OBRAS

[('ocê', 65),
 ('mecê', 40),
 ('saia', 32),
 ('Ocê', 31),
 ('meias', 12),
 ('Mecê', 10),
 ('meia', 8),
 ('Sossegue', 8),
 ('Negrinha', 5),
 ('malha', 4),
 ('saias', 4),
 ('ocês', 4),
 ('Ocês', 3),
 ('Excelentíssimo', 3),
 ('ia', 2),
 ('Sua', 2),
 ('mecês', 2),
 ('Fosse', 1),
 ('pensar', 1),
 ('lo', 1),
 ('íamos', 1),
 ('malhas', 1),
 ('sola', 1),
 ('Majestade', 1),
 ('Católica', 1),
 ('lui', 1),
 ('apertá', 1),
 ('dê', 1),
 ('dá', 1),
 ('pô', 1),
 ('Cópia', 1),
 ('Perorou', 1),
 ('Alteza', 1),
 ('Real', 1),
 ('Agorinha', 1)]

0 palavras que foram suprimidas de PALAVRAS

[]

2-51:
João Romão não saia nunca a passeio , nem ia à missa aos domingos ; tudo que rendia a sua venda e mais a quitanda seguia direitinho para a caixa econômica e daí entã

## Preparar material que será anotado pelo UDPipe

In [119]:
count_x85 = 0
for i in range(len(OBRAS_obras)):
        for s in range(len(OBRAS_obras[i]['sentences'])):
            for t in range(len(OBRAS_obras[i]['sentences'][s])):
                if OBRAS_obras[i]['sentences'][s][t]['forma'] == "\x85":
                    OBRAS_obras[i]['sentences'][s][t]['forma'] = OBRAS_obras[i]['sentences'][s][t]['forma'].replace("\x85", "__INDEF__")
                    count_x85 += 1
                    
print(f"{count_x85} caracteres especiais no arquivo foram substituidos por __INDEF__")

51 caracteres especiais no arquivo foram substituidos por __INDEF__


In [120]:
conllu = ""
for i in range(len(OBRAS_obras)):
    for s in range(len(OBRAS_obras[i]['sentences'])):
        conllu += "# sent_id = " + OBRAS_obras[i]['obra'][0].strip().replace(" ", "_") + "-" + str(i) + ":" + str(s)
        conllu += "\n# tituloobra = " + OBRAS_obras[i]['tituloobra'][0]
        conllu += "\n# autor = " + OBRAS_obras[i]['autor'][0]
        conllu += "\n# text = " + " ".join([x['forma'] for x in OBRAS_obras[i]['sentences'][s]])
        conllu += "\n" + "\n".join([x['forma'] + "\t" + "\t".join("_________") for x in OBRAS_obras[i]['sentences'][s]])
        conllu += "\n\n"

print("\n\n".join(conllu.split("\n\n")[:1]))
with open("corpoOBRAS_tokenizado.conllu", "w") as f:
    f.write(conllu)

# sent_id = A_Mortalha_de_Alzira_Prosa:romance_AA_1894_naturalismo_realismo_romantismo_masc-0:0
# tituloobra = A Mortalha de Alzira
# autor = Aluísio Azevedo
# text = A MORTALHA DE ALZIRA
A	_	_	_	_	_	_	_	_	_
MORTALHA	_	_	_	_	_	_	_	_	_
DE	_	_	_	_	_	_	_	_	_
ALZIRA	_	_	_	_	_	_	_	_	_


## Anotar com o UDPipe

In [ ]:
!python3 udpipe_vertical.py portuguese-bosque-ud-2.4-190531.udpipe corpoOBRAS_tokenizado.conllu corpoUDPIPE.conllu

Loading UDPipe model: done.


## Estrutura de dados do UDPIPE

In [ ]:
with open("corpoUDPIPE.conllu") as f:
    UDPIPE = f.read()
print("\n".join(UDPIPE.splitlines()[:9]))

In [89]:
UDPIPE_obras = []
obras_ja_adicionadas = []
sentences = []
for sentence in UDPIPE.split("\n\n"):
    if sentence:
        if not sentence.split("# tituloobra = ")[1].split("\n")[0] in obras_ja_adicionadas:
            if sentences:
                obras_ja_adicionadas.append(sentence.split("# tituloobra = ")[1].split("\n")[0])
                UDPIPE_obras.append({
                        'obra': sentence.split("# sent_id = ")[1].rsplit("-", 1)[0],
                        'tituloobra': sentence.split("# tituloobra = ")[1].split("\n")[0],
                        'autor': sentence.split("# autor = ")[1].split("\n")[0],
                        'sentences': sentences,
                    })
                sentences = []
            
        sentences += [[{
                    'forma': z.split("\t")[1],
                    'lema': z.split("\t")[2],
                    'pos': z.split("\t")[3],
                    'genero': z.split("\t")[5].split("Gender=")[1].split("|")[0] if "Gender=" in z.split("\t")[5] else "__INDEF__",
                } for z in sentence.splitlines() if len(z.split("\t")) > 5]]

print(len(PALAVRAS_obras))
print(len(OBRAS_obras))
print(len(UDPIPE_obras))

5
5
5


## Normalizar as formas de dizer o gênero

In [100]:
lista_genero_PALAVRAS = []
lista_genero_OBRAS = []
lista_genero_UDPIPE = []

generos_PALAVRAS = [c['genero'] for a in PALAVRAS_obras for b in a['sentences'] for c in b]
[lista_genero_PALAVRAS.append(x) for x in generos_PALAVRAS if not x in lista_genero_PALAVRAS]
print(lista_genero_PALAVRAS)

generos_OBRAS = [c['genero'] for a in OBRAS_obras for b in a['sentences'] for c in b]
[lista_genero_OBRAS.append(x) for x in generos_OBRAS if not x in lista_genero_OBRAS]
print(lista_genero_OBRAS)

generos_UDPIPE = [c['genero'] for a in UDPIPE_obras for b in a['sentences'] for c in b]
[lista_genero_UDPIPE.append(x) for x in generos_UDPIPE if not x in lista_genero_UDPIPE]
print(lista_genero_UDPIPE)

['F', '0', 'M', 'M/F', 'M/F+M', '+F', 'M+M/F', 'M/F+M/F', 'F+M/F', 'M/F+F', '+M']
['F', '0', 'M', 'M/F', 'M/F+M', '+F', 'M+M/F', 'M/F+M/F', 'F+M/F', 'M/F+F', '+M']
['Fem', '__INDEF__', 'Masc', 'Unsp']


In [108]:
#for a in range(len(OBRAS_obras)):
#    for b in range(len(OBRAS_obras[a]['sentences'])):
#        for c in range(len(OBRAS_obras[a]['sentences'][b])):
#            pass

generos_PALAVRAS = [c['genero'] for a in PALAVRAS_obras for b in a['sentences'] for c in b]
[lista_genero_PALAVRAS.append(x) for x in generos_PALAVRAS if not x in lista_genero_PALAVRAS]
print(lista_genero_PALAVRAS)

generos_OBRAS = [c['genero'] for a in OBRAS_obras for b in a['sentences'] for c in b]
[lista_genero_OBRAS.append(x) for x in generos_OBRAS if not x in lista_genero_OBRAS]
print(lista_genero_OBRAS)

generos_UDPIPE = [c['genero'] for a in UDPIPE_obras for b in a['sentences'] for c in b]
[lista_genero_UDPIPE.append(x) for x in generos_UDPIPE if not x in lista_genero_UDPIPE]
print(lista_genero_UDPIPE)

['M/F', 'M']
['M/F', 'M']
['M', 'M/F']
['M/F', 'M/F']
['M/F', 'M/F']
['M/F', 'M']
['M/F', 'M']
['M/F', 'M/F']
['M/F', 'M/F']
['F', 'M/F']
['M/F', 'M']
['M/F', 'F']
['M/F', 'F']
['M/F', 'M/F']
['M/F', 'M/F']
['M', 'M/F']
['M/F', 'M/F']
['M/F', 'F']
['M/F', 'M']
['F', 'M/F']
['M/F', 'F']
['M/F', 'M/F']
['M/F', 'F']
['M/F', 'M']
['M/F', 'F']
['M/F', 'M']
['M/F', 'M/F']
['M/F', 'M/F']
['M/F', 'M']
['M/F', 'M/F']
['M/F', 'F']
['M/F', 'M/F']
['M', 'M/F']
['M/F', 'M']
['M/F', 'M/F']
['M/F', 'M']
['M/F', 'M']
['M/F', 'M']
['M/F', 'M']
['M/F', 'M']
['M/F', 'M/F']
['M/F', 'M/F']
['M/F', 'M']
['M/F', 'F']
['M/F', 'M']
['M/F', 'M/F']
['M/F', 'M']
['M/F', 'F']
['M/F', 'M']
['M/F', 'M/F']
['M/F', 'F']
['F', 'M/F']
['M/F', 'M/F']
['M/F', 'M']
['M/F', 'M']
['M/F', 'M']
['M/F', 'F']
['M/F', 'M/F']
['M/F', 'F']
['M/F', 'M']
['M/F', 'M/F']
['F', 'M/F']
['F', 'M/F']
['M/F', 'M/F']
['M/F', 'M/F']
['F', 'M/F']
['F', 'M/F']
['M/F', 'M/F']
['M/F', 'F']
['M/F', 'F']
['M/F', 'M']
['M/F', 'F']
['M/F', 'M/F']
['M

## Criar HTML

In [ ]:
html = "<style>width:80%; margin:auto; padding-top:5px;</style>"
html += "<title>Comparação entre a anotação de nomes próprios - PALAVRAS e UDPipe</title>"
html += "<h1>Comparação entre a anotação de nomes próprios - PALAVRAS e UDPipe</h1><hr>"
html += "<table>"
html += "<tr><th>Metadados</th><th>corpoPALAVRAS</th><th>corpoUDPIPE</th><th>corpoOBRAS (golden)</th></td></tr>"
html += "<tr><td>Obras</td><td>{}</td><td>{}</td><td>{}</td></tr>".format(len(PALAVRAS_obras), len(UDPIPE_obras), len(OBRAS_obras))
html += "<tr><td>Sentenças</td><td>{}</td><td>{}</td><td>{}</td></tr>".format(sum([len(x['sentences']) for x in PALAVRAS_obras]), sum([len(x['sentences']) for x in UDPIPE_obras]), sum([len(x['sentences']) for x in OBRAS_obras]))
html += "<tr><td>Tokens</td><td>{}</td><td>{}</td><td>{}</td></tr>".format(sum([len(x) for y in PALAVRAS_obras for x in y['sentences']]), [len(x) for y in UDPIPE_obras for x in y['sentences']], [len(x) for y in OBRAS_obras for x in y['sentences']])
html += "<tr><td>Nomes próprios</td></tr>"
html += "</table>"
with open("gender-palavras-udpipe.html", "w") as f:
    f.write(html)